Для любых видео восстановить траекторию движения (t вектор). Выполнить визуализацию. 
Определить параметры которые влияют на "точность" определения вектора t
Использовать решение на базе нейронных сетей. 
Любые идеи. 

***slam прикрутить. 
Видео 1 но что бы работало.    

Анализ задания и пути решения.
Грузим из интернета качественное видео полета дрона. Сохраняем кусок не менее 2 мин.
Анализируем с помощью детектора sift.
Выводим траекторию на трехмерный график.
Результаты сохраняем в таблице csv.


In [41]:
# quit # reset all variables for repeated run of the program

# loading necessary libraries and two pictures
import cv2
import numpy as np
import matplotlib.pyplot as plt

def filter_matches_distance(matches, dist_threshold):
    """
    Filter matched features from two images by distance between the best matches

    Arguments:
    match -- list of matched features from two images
    dist_threshold -- maximum allowed relative distance between the best matches, (0.0, 1.0) 

    Returns:
    filtered_match -- list of good matches, satisfying the distance threshold
    """
    filtered_match = []
    for m, n in matches:
        if m.distance <= dist_threshold * n.distance:
            filtered_match.append(m)

    return filtered_match

Согласно информации из лекции, это не будет работать если:
- Небольшое перемещение камеры (должно быть минимум несколько пикселей)
- Не исправленное изображение
- Мало точек или точки все сосредоточены в одной области

Поэтому мы далее анализируем не соседние кадры, а через N (например, 5 или 10) кадров. Учитывая фреймрейт 30 кадров в секунду, это вполне еще ВЧ дискретизация по времени.

In [42]:
def match_features(des1, des2, matching='BF', detector='sift', sort=True, k=2):
    """
    Match features from two images

    Arguments:
    des1 -- list of the keypoint descriptors in the first image
    des2 -- list of the keypoint descriptors in the second image
    matching -- (str) can be 'BF' for Brute Force or 'FLANN'
    detector -- (str) can be 'sift or 'orb'. Default is 'sift'
    sort -- (bool) whether to sort matches by distance. Default is True
    k -- (int) number of neighbors to match to each feature.

    Returns:
    matches -- list of matched features from two images. Each match[i] is k or less matches for 
               the same query descriptor
    """
    if matching == 'BF':
        if detector == 'sift':
            matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck=False)
        elif detector == 'orb':
            matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING2, crossCheck=False)
        matches = matcher.knnMatch(des1, des2, k=k)
    elif matching == 'FLANN':
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
        matches = matcher.knnMatch(des1, des2, k=k)
    
    if sort:
        matches = sorted(matches, key = lambda x:x[0].distance)

    return matches

Загружаем видео и считываем попарно кадр и следующий кадр. Рассчитываем позицию камеры, если номер следующего считанного кадра кратен десяти.

In [ ]:

# video_path = 'vid.mp4'  # file name (and path if necessary)
video_path = 'narach.mp4'  # file name (and path if necessary)
myvid = cv2.VideoCapture(video_path) # loading into cv2
# Check if video opened successfully
if not myvid.isOpened():
     print("Video file not found")
else:
     print("Video is opened") 

# camera matrix K
K = np.array([[3000 ,   0.    , 960],
       [  0.    , 3000 , 540 ],
      [  0.    ,   0.    ,   1.]], dtype=np.float32)

sift = cv2.SIFT_create()
poses = [] # list for camera poses
# List for camera positions
positions = [np.array([0, 0, 0])]

# Reading the first frame of the video
ret, frame = myvid.read()
img1 = cv2. cvtColor(frame, cv2. COLOR_BGR2GRAY)
i = 0
interval = 5 # interval between frames considered, e.g. interval = 5 for each 5th frame to be taken into account
while True: # i < 1000:
     i += 1
     ret, frame = myvid.read() # read next frame          
     if ret and i % interval == 0: # each 10-th frame
         print(f'Current frame is {i}')
         img2 = cv2. cvtColor(frame, cv2. COLOR_BGR2GRAY)
         try:
             keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
             keypoints2, descriptors2 = sift.detectAndCompute(img2, None)      
             matches = match_features(descriptors1, descriptors2, matching = 'BF', detector = 'sift', sort = True)
             matches = filter_matches_distance(matches, 0.7)
             # matches = matcher.match(np.array(descriptors1,dtype=np.uint8), np.array(descriptors2,dtype=np.uint8))
             pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
             pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
       
             F, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.LMEDS, prob = 0.999, threshold = 1.0) # LMEDS    
             # F, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob = 0.999, threshold = 1.0) #RANSAC    
             a, R, t, b = cv2.recoverPose(F, pts1, pts2, K, mask)
             # print(R)
             poses.append((R, t))

             # Обновить позицию камеры
             current_position = positions[-1]
             new_position = current_position + np.dot(R, t).T[0]
             positions.append(new_position)
             # Saving current frame as previous one
             img1 = img2 
         except:
              pass
     elif ret and i % interval != 0:
          pass
     else:
          print('The video is over')
          myvid.release()          # releasing video channel after viewing is over
          # cv2.destroyAllWindows()  # close all windows
          break                    # break the cycle


Video is opened
Current frame is 5
Current frame is 10
Current frame is 15
Current frame is 20
Current frame is 25
Current frame is 30
Current frame is 35
Current frame is 40
Current frame is 45
Current frame is 50
Current frame is 55
Current frame is 60
Current frame is 65
Current frame is 70
Current frame is 75
Current frame is 80
Current frame is 85
Current frame is 90
Current frame is 95
Current frame is 100
Current frame is 105
Current frame is 110
Current frame is 115
Current frame is 120
Current frame is 125
Current frame is 130
Current frame is 135
Current frame is 140
Current frame is 145
Current frame is 150
Current frame is 155
Current frame is 160
Current frame is 165
Current frame is 170
Current frame is 175
Current frame is 180
Current frame is 185
Current frame is 190
Current frame is 195
Current frame is 200
Current frame is 205
Current frame is 210
Current frame is 215
Current frame is 220
Current frame is 225
Current frame is 230
Current frame is 235
Current frame is 

Визуализируем результат


In [44]:
print(poses)

[(array([[ 9.99998955e-01, -4.56662280e-04,  1.37141661e-03],
       [ 4.57904506e-04,  9.99999485e-01, -9.05620328e-04],
       [-1.37100234e-03,  9.06247360e-04,  9.99998650e-01]]), array([[-0.48763349],
       [-0.0354462 ],
       [ 0.87232858]])), (array([[ 0.99997972, -0.00106224,  0.00627896],
       [ 0.00107229,  0.99999815, -0.00159831],
       [-0.00627725,  0.00160501,  0.99997901]]), array([[ 0.98119254],
       [-0.18401272],
       [-0.05831391]])), (array([[ 0.99995569, -0.00121749,  0.00933478],
       [ 0.00123165,  0.9999981 , -0.00151133],
       [-0.00933292,  0.00152276,  0.99995529]]), array([[ 0.99035845],
       [-0.13789556],
       [-0.01322681]])), (array([[ 0.99982472, -0.00231375,  0.01857913],
       [ 0.0022863 ,  0.99999626,  0.00149835],
       [-0.01858252, -0.00145561,  0.99982627]]), array([[-0.99283011],
       [-0.09831942],
       [-0.0679828 ]])), (array([[ 0.99990103, -0.00153645,  0.01398431],
       [ 0.0014765 ,  0.99998968,  0.00429594],
  

In [45]:
# F, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.LMEDS, prob=0.999, threshold=1.0)
# a, R, t, b = cv2.recoverPose(F, pts1, pts2, K, mask)

In [46]:
# poses

In [47]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def create_trajectory(poses):
    trajectory = [np.array([0, 0, 0])]
    current_pose = np.eye(4)
    pose_cam = [np.array([[0, 0, 0],[0, 0, 0],[0, 0, 0]])]

    for R, t in poses:
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = t.T
        current_pose = np.dot(current_pose, T)
        trajectory.append(current_pose[:3, 3])
        pose_cam.append(current_pose[:3, :3])

    return np.array(trajectory),np.array(pose_cam)

In [48]:
trajectory, pose_cam = create_trajectory(poses)

In [49]:
import plotly.graph_objects as go
import numpy as np
# Создание фигуры
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=trajectory[1:, 0],
    y=trajectory[1:, 1],
    z=trajectory[1:, 2],
    mode='lines+markers',
    marker=dict(size=3, color='blue'),
    line=dict(color='blue', width=2),
    name='Camera Trajectory'
))

# Добавление ориентации камеры в каждой точке траектории
for i, (R, t) in enumerate(zip(pose_cam,trajectory)):
    # Направление камеры (ось Z камеры)
    camera_direction = R @ np.array([0, 0, 1])  # Направление оси Z камеры
    camera_direction_end = t + camera_direction * 0.5  # Конец вектора направления

    # Добавляем линию, представляющую направление камеры
    fig.add_trace(go.Scatter3d(
        x=[t[0], camera_direction_end[0]],
        y=[t[1], camera_direction_end[1]],
        z=[t[2], camera_direction_end[2]],
        mode='lines',
        line=dict(color='green', width=2),
        name=f'Camera Direction {i}' if i == 0 else None,
        showlegend=False if i > 0 else True
    ))

# Настройка макета графика
fig.update_layout(
    title='Camera Motion Trajectory',
    scene = dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis'
    ),
    showlegend=True
)

# Показать график
fig.show()

In [50]:
fig.write_html("camera_trajectory_sift.html")

Выводы: с  помощью детектора sift проанализировали видеролик и рассчитали 3D-траекторию движения дрона и ориентацию его камеры.
Для нахождения эссенциальной матрицы использовался метод cv2.LMEDS и cv2.RANSAC.
На точность расчета в нашем случае влияет размер промежутка между кадрами, принимаемыми во внимание, и метод расчет эссенциальной матрицы.
Так, если в рассмотрение принимался каждый пятый кадр, траектория выглядела неестественно изорванной.
При рассмотрении каждого десятого кадра траектория выглядит плавной.

---